In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from scipy import stats
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [59]:
data = pd.read_csv('normalised.csv', index_col= 0)
#data = pd.read_csv('scaled.csv', index_col= 0)

# Define a threshold for the Z-score (e.g., 3 or -3 for a typical threshold)
threshold = 3

# Initialize a dictionary to store the outliers for each column
outliers = {}

# Iterate through each column and detect outliers using the Z-score
for column in data.columns:
    z_scores = np.abs(stats.zscore(data[column]))
    column_outliers = np.where(z_scores > threshold)
    outliers[column] = column_outliers

# Print the outliers for each column
for column, column_outliers in outliers.items():
    print(f"Column '{column}' has {len(column_outliers[0])} outliers:")
    print(data.iloc[column_outliers[0]])

# You can further process or handle the outliers as needed


Column 'ExternalRiskEstimate' has 19 outliers:
      ExternalRiskEstimate  MSinceOldestTradeOpen  MSinceMostRecentTradeOpen  \
1243              0.505425               0.262309                   0.006398   
1506              0.478235               0.127126                   0.006054   
2365              0.510601               0.196385                   0.039277   
2524              0.520391               0.175481                   0.121021   
2959              0.475121               0.150038                   0.050013   
3409              0.479272               0.273003                   0.060667   
3450              0.516491               0.072068                   0.054051   
4087              0.520447               0.276693                   0.046116   
4342              0.477194               0.286317                   0.006363   
5836              0.495194               0.403065                   0.051823   
6419              0.475004               0.392395                   0.000

should we worry about these outliers? ^^^


In [60]:
y.value_counts()


RiskPerformance
1                  5128
0                  4733
dtype: int64

### Split data

In [61]:
# Read csv
#data = pd.read_csv('normalised.csv', index_col= 0)
data = pd.read_csv('scaled.csv', index_col= 0)
y = pd.read_csv('target.csv', index_col= 0)

# adjust train, test, validation size as you wish
X_train, X_temp, y_train, y_temp = train_test_split(data, y, test_size=0.25, random_state=42)

X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.40, random_state=42)


In [62]:
# ogistic regression model
model = LogisticRegression(random_state=42)

# Train the model 
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate 
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)


Accuracy: 0.7436676798378926
Confusion Matrix:
 [[332 135]
 [118 402]]
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.71      0.72       467
           1       0.75      0.77      0.76       520

    accuracy                           0.74       987
   macro avg       0.74      0.74      0.74       987
weighted avg       0.74      0.74      0.74       987



/Users/samblackburn/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/samblackburn/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [63]:
model = Sequential()

# Add input layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
# Hidden layers
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))  # Dropout layer to reduce overfitting
# output layer with sigmoid function
model.add(Dense(units=1, activation='sigmoid'))
# Compile with loss function, optimizer, and evaluation metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_validation, y_validation), verbose =0)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


31/31 [==============================] - 0s 546us/step - loss: 0.5216 - accuracy: 0.7467
Test Loss: 0.5216
Test Accuracy: 0.7467


In [65]:
model2 = Sequential()
model2.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
model2.add(Dense(units=64, activation='relu'))
model2.add(Dense(units=32, activation='relu'))
model2.add(Dense(units=16, activation='tanh'))
model2.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_validation, y_validation), verbose =0)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

31/31 [==============================] - 0s 460us/step - loss: 0.5263 - accuracy: 0.7396
Test Loss: 0.5263
Test Accuracy: 0.7396


In [66]:
#Adding Dropout for regularisation
model3 = Sequential()
model3.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model3.add(Dropout(0.3))
model3.add(Dense(units=32, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(units=16, activation='relu'))
model3.add(Dropout(0.1))
model3.add(Dense(units=1, activation='sigmoid'))

custom_optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])



model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_validation, y_validation), verbose =0)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


31/31 [==============================] - 0s 434us/step - loss: 0.5288 - accuracy: 0.7376
Test Loss: 0.5288
Test Accuracy: 0.7376
